In [ ]:
# KMeans Above Phonemes

In [ ]:
### Collect training features

In [15]:
import gc
import numpy as np
np.random.seed(0)
import random
random.seed(0)
import os, sys
import matplotlib.pyplot as plt

directory = os.path.abspath('/Users/joris/Documents/Work/bsc ai/bt/Bachelor-Thesis/code')
sys.path.append(directory)

from lib.conceptors import *
from lib.esn import ESN
from lib.helpers import *
from dataset.loading import Feature_Collector

import warnings
warnings.filterwarnings("ignore")

XorZ = "X"

print("starting")

path = '../../data/'
fc = Feature_Collector(path)

dr = []
speakers = []

long_version = False
n_mels = 13
delta = False
delta_delta = False
subsamples = 10

path_option = "Final"+str(long_version)+str(n_mels)+str(delta)+str(delta_delta)+str(subsamples)

if dr:
    path_option = str(dr)+"_"+path_option
if len(speakers):
    path_option = str(speakers[0])+"_"+path_option

features_train, labels_train, _ = fc.collectFeaturesInSegments(
    n_mels=n_mels, delta=delta, delta_delta=delta_delta,
    long_version=long_version, speakers=speakers, dr=dr,
    subsamples=subsamples, path_option=path_option)

print(f"Loaded to {len(features_train)} samples of shape {features_train[0].shape}")

gc.collect()

-from output
---- success


247

In [16]:
# init reservoir
import random
random.seed(0)
np.random.seed(0)
t_washout = 0 # number of washout steps
aperture = 5
nb_trials = 1

esn_params = {
    "in_dim": n_mels*(1+delta+delta_delta),
    "out_dim": n_mels*(1+delta+delta_delta),
    "N": 10,
    "W_in_scale": 1.5,
    "b_scale": .2,
    "spectral_radius": 1.5
}

esn = ESN(esn_params)

In [17]:
group = {}
for i in range(len(features)):
    if labels[i] not in group.keys():
        group[labels[i]] = []
    group[labels[i]].append(features[i])
group2 = {}
# Cut down to same number of samples
min_samples = 10

data = []
labels = []
for x,y in group.items():
    if x in ['ih', 'ah', 'ae', 'iy', 'aa', 'eh', 'ey', 'ay', 'uh', 'oy'] + ['t', 'b', 'd', 'p', 'dx', 'k', 'dh', 'g']:
        group2[x] = random.sample(y, min_samples)
        data += group2[x]
        labels += [x]*min_samples
group = group2

phonemes = list(group.keys())
print(min_samples)
print(str(len(phonemes))+" phonemes")
#[print(x," ss ",len(y)) for x,y in group.items() ]

10
18 phonemes


In [22]:
def assign_to_clusters(nb_points, nb_clusters):
    assignments = [ [] for _ in range(nb_clusters) ]
    points = [ x for x in range(nb_points) ]
    np.random.shuffle(points)
    for i in range(nb_clusters):
        assignments[i] = points[i*int(nb_points/nb_clusters):(i+1)*int(nb_points/nb_clusters)]
    return assignments

# K-means
def kmeans(data, nb_conceptors, aperture, max_epochs=100):
    # Initial assignments and initial conceptors
    nb_points = data.shape[0]
    new_assignments = assign_to_clusters(nb_points, nb_conceptors)
    # Training loop
    for epoch in range(max_epochs):
        print("epoch:",epoch)
        # recompute centroids based on subset of assigned state
        Cs = [ [] for _ in range(nb_conceptors) ]
        for i, assignments in enumerate(new_assignments):
            Z = np.array([])
            for x in assignments:
                z = esn.run(data[x].T)
                Z = np.hstack([Z, z]) if Z.size else z
            Cs[i] = compute_c(Z, aperture)
        Ns = Ns_from_Cs(Cs)
        print("- optimizing +")
        Cs = optimize_apertures(Cs)
        print("- optimizing -")
        Ns = optimize_apertures(Ns)
        
        # recompute assignments by find the closest conceptor for each of the state points
        old_assignments = new_assignments.copy()
        new_assignments = [ [] for _ in range(nb_conceptors) ]
        
        Es = np.zeros((nb_conceptors, nb_points))
        for t in range(nb_points):
            Es[:,t] = np.array( evidences_for_Cs_z(X[:,t], Cs, Ns) )
        
        for t in range(nb_points):
            conceptor_index = np.argmax(Es[:,t])
            new_assignments[ conceptor_index ].append(t)

        # stop if converged
        for new_assignment, old_assignment in zip(new_assignments, old_assignments):
            if set(new_assignment) == set(old_assignment):
                print("Converged")
                return Cs, new_assignments

    return Cs, new_assignments

#######################################
Cs_kmeans, assignments_kmeans = kmeans(data, nb_conceptors=2, aperture=aperture, max_epochs=100)


epoch: 0
(90, 4, 10)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 10)